## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

### Import our existing Weather Database

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Clyde River,CA,70.4692,-68.5914,-4.00,77,90,10.36,overcast clouds
1,1,Punta Arenas,CL,-53.1500,-70.9167,51.80,46,0,6.91,clear sky
2,2,New Norfolk,AU,-42.7826,147.0587,55.99,59,68,1.01,broken clouds
3,3,Rikitea,PF,-23.1203,-134.9692,79.20,73,3,15.55,clear sky
4,4,Jamestown,US,42.0970,-79.2353,42.01,75,90,22.82,overcast clouds


### Request the user preferences, filter the city choices, and scrub the data

In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
print("Welcome to Vacation Search.  Let's get started on planning your trip!")
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

Welcome to Vacation Search.  Let's get started on planning your trip!
What is the minimum temperature you would like for your trip? 73
What is the maximum temperature you would like for your trip? 88.9


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
selected_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
selected_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Rikitea,PF,-23.1203,-134.9692,79.20,73,3,15.55,clear sky
11,11,Atuona,PF,-9.8000,-139.0333,80.91,74,98,17.13,overcast clouds
18,18,Kapaa,US,22.0752,-159.3190,77.00,65,90,18.41,light rain
19,19,Arraial Do Cabo,BR,-22.9661,-42.0278,80.60,83,0,8.05,clear sky
20,20,Vaini,TO,-21.2000,-175.2000,84.20,79,40,10.36,scattered clouds
21,21,Acapulco,MX,16.8634,-99.8901,86.00,62,1,9.22,clear sky
33,33,Rio De Janeiro,BR,-22.9028,-43.2075,84.20,89,0,12.66,clear sky
47,47,Dingle,PH,10.9995,122.6711,78.17,85,100,6.38,overcast clouds
52,52,Siguatepeque,HN,14.6000,-87.8333,75.20,41,20,9.22,few clouds
56,56,Pangai,TO,-19.8000,-174.3500,84.20,79,20,17.27,few clouds


In [15]:
# 4a. Determine if there are any empty rows.
selected_cities_df.count()

City_ID                228
City                   228
Country                228
Lat                    228
Lng                    228
Max Temp               228
Humidity               228
Cloudiness             228
Wind Speed             228
Current Description    228
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# Skip as no rows to drop. You would use this code if you needed to remove empty rows
# clean_df = selected_cities_df.dropna()

### Create the hotel df and use Google Directions API to find hotels in desired locations

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = selected_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Rikitea,PF,79.20,clear sky,-23.1203,-134.9692,
11,Atuona,PF,80.91,overcast clouds,-9.8000,-139.0333,
18,Kapaa,US,77.00,light rain,22.0752,-159.3190,
19,Arraial Do Cabo,BR,80.60,clear sky,-22.9661,-42.0278,
20,Vaini,TO,84.20,scattered clouds,-21.2000,-175.2000,
21,Acapulco,MX,86.00,clear sky,16.8634,-99.8901,
33,Rio De Janeiro,BR,84.20,clear sky,-22.9028,-43.2075,
47,Dingle,PH,78.17,overcast clouds,10.9995,122.6711,
52,Siguatepeque,HN,75.20,few clouds,14.6000,-87.8333,
56,Pangai,TO,84.20,few clouds,-19.8000,-174.3500,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame and add it to the params dictionary
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... skipping")    

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


### Drop missing hotels and save the hotel data to a csv

In [22]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]

In [24]:
# confirm clean_hotel_df contents
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Rikitea,PF,79.20,clear sky,-23.1203,-134.9692,Pension Maro'i
11,Atuona,PF,80.91,overcast clouds,-9.8000,-139.0333,Villa Enata
18,Kapaa,US,77.00,light rain,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
19,Arraial Do Cabo,BR,80.60,clear sky,-22.9661,-42.0278,Pousada Porto Praia
20,Vaini,TO,84.20,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
21,Acapulco,MX,86.00,clear sky,16.8634,-99.8901,HS Hotsson Smart Acapulco
33,Rio De Janeiro,BR,84.20,clear sky,-22.9028,-43.2075,Hotel Málaga
47,Dingle,PH,78.17,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY
52,Siguatepeque,HN,75.20,few clouds,14.6000,-87.8333,Orquidea Limon Suites
56,Pangai,TO,84.20,few clouds,-19.8000,-174.3500,Ha'apai Beach Resort


In [27]:
# 8a. Create the output File (CSV)
# we are running this notebook in the same folder where we want to save it.
output_data_file = './WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

### Customize and use Google Maps to create Vacation results map 

In [33]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
#    Assumed the hotel name was also desired even though the intructions excluded it (it is in the final image)
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [47]:
# 11a. Add a marker layer for each city to the map. 
#center locations
# center=(30.0, 31.0) - historical geographic center
# cneter=(40.75, -74.00) - NYC, the true center of the world
fig = gmaps.figure(center=(40.75, -74.00), zoom_level=3.0)
heat_layer = gmaps.heatmap_layer(locations)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))